# Resting state analysis

In [8]:
import pickle
from pathlib import Path
import os

import mne

import numpy as np
import scipy.stats

import matplotlib as mpl
import matplotlib.pyplot as plt
try:
    %matplotlib inline
except:
    pass

import pandas as pd

from tqdm import tqdm as tqdm


## Take the raw data and frequency-transform it

In this package I have already made scripts that effectively turn the CSV files into raw files using a generator function: `restraws()`. You can find that [here](data/processed/resting.py). It doesn't do anything other than: 1) load the data from CSV; 2) load the channel locations; 3) combine the two to make a neat [`Raw`](http://martinos.org/mne/dev/generated/mne.io.Raw.html) data with a proper montage, all zero-channels set as `bads`, and the subject's name in the info structure. The neat thing is that it does it _on each iteration_, so it loads the data one by one, and removes it from memory when the next one comes up.

Here, I'm going to load that data, then average-ref it and do a time-frequency transform on it. This is already effectively preprocessed. The time-frequency data is saved to the data/interim/freqanalysis.

Note that this step takes ages, and if you have already done it, your data should be in `data/interim/freqanalysis`. If that's the case, [skip to the next section.](#Load-the-data-back-from-pickle)

In [ ]:
from data.preprocessed import resting

for raw in resting.raws():
    
    # output names
    psdfname = (Path('.') / 'data' / 'interim' / 'freqanalysis' /
                ('rest-' + raw.info['subject_info'] + '.pickle'))
    infofname = (Path('.') / 'data' / 'interim' / 'info' /
                 ('rest-' + raw.info['subject_info'] + '.pickle'))
    
    # save the info structure to file (incl bads)
    with open(infofname, 'wb+') as f:
        pickle.dump(raw.info, f)

    # if it exists already, skip
    if os.path.isfile(psdfname): continue

    # try cropping it; this will fail if less than 200s; so skip
    try:
        raw.crop(tmin=0, tmax=200)
    except ValueError:
        continue

    # average reference
    raw.set_eeg_reference()
    raw.apply_proj()
    
    # do the time-frequency analysis
    psd, freqs = mne.time_frequency.psd_multitaper(raw, fmin=1, fmax=30, n_jobs=4)
    
    # save TFR and Info to pickle
    with open(psdfname, 'wb+') as f:
        pickle.dump((freqs, psd), f)



## Load the data back from pickle

In `data/interim/freqanalysis` [another python script](data/interim/freqanalysis/__init__.py) sits that lets you load the data __back__ into memory via a generator. So here I do that, then fit the linear regression in semilog and loglog space.

In [30]:
from data.interim.freqanalysis import psds
import scipy.stats

alldata = []
semilogslopes = []
semilogintercepts = []
pids = []

for pid, freq, psd in psds():
    
    pids.append(pid)
    
    # identify the right frequency band
    idx = ((freq > 4) & (freq < 7)) | ((freq > 14) & (freq < 24))
    
    # fit linear regression in semilog space
    semilogfits = [scipy.stats.linregress(freq[idx], np.log10(psd.T[idx, sensor]))
                      for sensor in range(psd.shape[0])]
    
    semilogfilter = [(fit.slope < 0) &
                     (np.abs(fit.intercept - np.mean([f.intercept for f in semilogfits])) <
                      3 * np.std([f.intercept for f in semilogfits]))
                     for fit in semilogfits]
    
    loglogfits  = [scipy.stats.linregress(np.log10(freq[idx]), np.log10(psd.T[idx, sensor]))
                      for sensor in range(psd.shape[0])]
    
    
    # make numpy arrays with the relevant info
    semilogslopes.append(np.mean([fit.slope for fit, filt in zip(semilogfits, semilogfilter) if filt]))
    semilogintercepts.append(np.mean([fit.intercept for fit, filt in zip(semilogfits, semilogfilter) if filt]))

semilogslopes = np.array(semilogslopes)
semilogintercepts = np.array(semilogintercepts)

alldata = pd.DataFrame({'semilogslopes': semilogslopes,
                        'semilogintercepts': semilogintercepts},
                       index=pids)


## Load the phenotypic data & combine with data so far

Since most of the stats analysis will be done in an R-notebook, we're going to merge the data we have so far and save it to csv so that we can then read it into R.

The data will be saved to [`data/interim/csv/metrics-pheno.csv`](data/interim/csv/metrics-pheno.csv)

In [38]:
from data import phenotypes

phenotypes = phenotypes.set_index('EID')

alldf = pd.DataFrame(alldata).join(phenotypes, how='inner')
alldf.index.name = 'pid'

# save to csv
alldf.to_csv('data/interim/csv/metrics-pheno.csv')

## Stats Analysis

Here we're going to do some stats on this thing.


In [50]:
# load the data from file
alldf = pd.read_csv('data/interim/csv/metrics-pheno.csv')


---

### Correlations

Simple correlations between age, slope and intercept

In [54]:
import scipy.stats

print(scipy.stats.linregress(alldf['Age'], alldf['semilogintercepts']))
print(scipy.stats.linregress(alldf['Age'], alldf['semilogslopes']))
print(scipy.stats.linregress(alldf['semilogslopes'], alldf['semilogintercepts']))

LinregressResult(slope=-0.069901807210851027, intercept=3.7853176823839196, rvalue=-0.51692053749096056, pvalue=4.6454619255008044e-15, stderr=0.0082266443861644702)
LinregressResult(slope=0.0018916318243253081, intercept=-0.085969941896888238, rvalue=0.37894504011075753, pvalue=3.1433596619017633e-08, stderr=0.00032829652473793534)
LinregressResult(slope=-13.995411675614438, intercept=2.1351099622073941, rvalue=-0.51663217024142305, pvalue=4.8388953746600994e-15, stderr=1.6483545838897606)


---

### Mediation Check

Is there a link between age and slope, and if so is it mediated by intercept?

In [49]:
import statsmodels.api as sm
from statsmodels.stats.mediation import Mediation

outcome_model = sm.GLM.from_formula("semilogslopes ~ Age + semilogintercepts", alldf)

mediator_model = sm.GLM.from_formula("semilogintercepts ~ Age", alldf)

mediation = Mediation(outcome_model, mediator_model, 'Age', 'semilogintercepts')

results = mediation.fit(method='bootstrap', n_rep=1000)

results.summary()

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.001154,-0.000169,0.002586,0.0824
ACME (treated),0.001154,-0.000169,0.002586,0.0824
ADE (control),0.000754,0.000002,0.001527,0.0500
ADE (treated),0.000754,0.000002,0.001527,0.0500
Total effect,0.001908,0.000404,0.003386,0.0124
Prop. mediated (control),0.594691,-0.152263,1.027467,0.0700
Prop. mediated (treated),0.594691,-0.152263,1.027467,0.0700
ACME (average),0.001154,-0.000169,0.002586,0.0824
ADE (average),0.000754,0.000002,0.001527,0.0500
Prop. mediated (average),0.594691,-0.152263,1.027467,0.0700
